# Imports & Setup

In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import os
import streamlit as st
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import requests
from io import StringIO
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [55]:
load_dotenv()

True

# Data Preprocessing

## Declarations and Functions

In [56]:
# Sets and Parameters

request_set = ['TIME_SERIES_DAILY', 'RSI', 'NEWS_SENTIMENT', 'REAL_GDP']
sa_request_parameters = ['technology', 'retail_wholesale']
premium_request_set = ['MACD']

queries = {
    'TIME_SERIES_DAILY':'https://www.alphavantage.co/query?function={}&symbol={}&outputsize={}&datatype={}&apikey={}',
    'NEWS_SENTIMENT': 'https://www.alphavantage.co/query?function={}&tickers={}&limit={}&apikey={}',
    'RSI': 'https://www.alphavantage.co/query?function={}&symbol={}&interval={}&time_period={}&series_type={}&datatype={}&apikey={}',
    'REAL_GDP': 'https://www.alphavantage.co/query?function={}&interval={}&apikey={}'
}

In [57]:
# Data Import Function

def alpha_multicall(request_set, symbol: str=None, datatype='json', outputsize='compact',
                     limit=100, interval='daily', gdp_interval='quarterly', time_period=14, series_type='close'):
    dataframes = []
    # The loop that looks through the request set and makes an API call for each one:
    if datatype == 'json':
        for request_type in request_set:
            if request_type == 'TIME_SERIES_DAILY':
                df = pd.DataFrame(requests.get(f'{queries[request_type]}'.format(request_type, symbol, outputsize, datatype, os.getenv('ALPHAVANTAGE_API_KEY'))).json())
                dataframes.append(df)
            elif request_type == 'RSI':
                df = pd.DataFrame(requests.get(f'{queries[request_type]}'.format(request_type, symbol, interval, time_period, series_type, datatype, os.getenv('ALPHAVANTAGE_API_KEY'))).json())
                dataframes.append(df)
            elif request_type == 'NEWS_SENTIMENT':
                df = pd.DataFrame(requests.get(f'{queries[request_type]}'.format(request_type, symbol, limit, os.getenv('ALPHAVANTAGE_API_KEY'))).json())
                dataframes.append(df)
            elif request_type == 'REAL_GDP':
                df = pd.DataFrame(requests.get(f'{queries[request_type]}'.format(request_type, symbol, datatype, os.getenv('ALPHAVANTAGE_API_KEY'))).json())
                dataframes.append(df)
    else:
        print('A data type outside of JSON isn\'t support at the moment.')
        return None
    # Returns a tuple so we can unpack all the returned DataFrames into separate objects:
    return tuple(dataframes)

## First Set

In [58]:
aapl_tsd, aapl_rsi, aapl_ns, gdp = alpha_multicall(request_set, 'AAPL')

In [64]:
aapl_tsd.head(20)

,Meta Data,Time Series (Daily)
1. Information,"Daily Prices (open, high, low, close) and Volumes",NaN
2. Symbol,AAPL,NaN
3. Last Refreshed,2024-02-26,NaN
4. Output Size,Compact,NaN
5. Time Zone,US/Eastern,NaN
2024-02-26,NaN,"{'1. open': '182.2400', '2. high': '182.7600',..."
2024-02-23,NaN,"{'1. open': '185.0100', '2. high': '185.0400',..."
2024-02-22,NaN,"{'1. open': '183.4800', '2. high': '184.9550',..."
2024-02-21,NaN,"{'1. open': '181.9400', '2. high': '182.8888',..."
2024-02-20,NaN,"{'1. open': '181.7900', '2. high': '182.4300',..."


In [60]:
aapl_rsi.head()

,Meta Data,Technical Analysis: RSI
1: Symbol,AAPL,NaN
2: Indicator,Relative Strength Index (RSI),NaN
3: Last Refreshed,2024-02-26,NaN
4: Interval,daily,NaN
5: Time Period,14,NaN


In [61]:
aapl_ns.head()

,items,sentiment_score_definition,relevance_score_definition,feed
0,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Berkshire Hathaway's march toward $...
1,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...","{'title': 'Futures: Bitcoin Leader Soars, But ..."
2,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Arm Stock: Hot Run For Chip Designe...
3,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Dow Jones index struggling for rele...
4,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",{'title': 'Bill Gates Considers Himself To Be ...


In [62]:
gdp.head()

,name,interval,unit,data
0,Real Gross Domestic Product,annual,billions of dollars,"{'date': '2023-01-01', 'value': '22375.307'}"
1,Real Gross Domestic Product,annual,billions of dollars,"{'date': '2022-01-01', 'value': '21822.037'}"
2,Real Gross Domestic Product,annual,billions of dollars,"{'date': '2021-01-01', 'value': '21407.692'}"
3,Real Gross Domestic Product,annual,billions of dollars,"{'date': '2020-01-01', 'value': '20234.074'}"
4,Real Gross Domestic Product,annual,billions of dollars,"{'date': '2019-01-01', 'value': '20692.087'}"


## Second Set